In [ ]:
GBRT_DIR_PATH = '/home/pereldamian/PycharmProjects/gbrt_new'
TRAIN_DATA_PATH = './data_liberty/train.csv'
LABEL_NAME = 'Hazard'

In [1]:
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import operator
import warnings
warnings.filterwarnings('ignore')

# GBRT imports
sys.path.insert(0, GBRT_DIR_PATH)
from data_utils import parse_data
from gbrt_algorithm import gbrt
from feature_selection import ensemble_feature_importance
from main import HParams

%matplotlib inline

In [ ]:
train_dataset, test_dataset = parse_data(TRAIN_DATA_PATH, label_name=LABEL_NAME)

In [3]:
train_dataset.data.head(5)

,Hazard,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,T1_V9,...,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
Id,,,,,,,,,,,,,,,,,,,,,
1,1,15,3,2,N,B,N,B,B,D,...,2,37,1,11,6,Y,N,E,2,2
2,4,16,14,5,H,B,N,B,B,C,...,2,22,1,18,5,Y,Y,E,2,1
3,1,10,10,5,N,K,N,B,B,E,...,6,37,2,14,6,Y,Y,E,6,1
4,1,18,18,5,N,K,N,B,B,E,...,2,25,1,1,6,Y,N,C,2,6
5,1,13,19,5,N,H,N,B,B,E,...,1,22,1,2,7,N,N,E,1,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50999 entries, 1 to 101999
Data columns (total 33 columns):
Hazard    50999 non-null int64
T1_V1     50999 non-null int64
T1_V2     50999 non-null int64
T1_V3     50999 non-null int64
T1_V4     50999 non-null object
T1_V5     50999 non-null object
T1_V6     50999 non-null object
T1_V7     50999 non-null object
T1_V8     50999 non-null object
T1_V9     50999 non-null object
T1_V10    50999 non-null int64
T1_V11    50999 non-null object
T1_V12    50999 non-null object
T1_V13    50999 non-null int64
T1_V14    50999 non-null int64
T1_V15    50999 non-null object
T1_V16    50999 non-null object
T1_V17    50999 non-null object
T2_V1     50999 non-null int64
T2_V2     50999 non-null int64
T2_V3     50999 non-null object
T2_V4     50999 non-null int64
T2_V5     50999 non-null object
T2_V6     50999 non-null int64
T2_V7     50999 non-null int64
T2_V8     50999 non-null int64
T2_V9     50999 non-null int64
T2_V10    50999 non-null int64
T2_V11   

# Model Selection

Training models with different hyper parameters.
All models logs are saved in models_data dictionary.

In [ ]:
models_data = {}

params_list = []

weight_decay_list = [0.1, 0.3, 0.6, 0.9]
for weight_decay in weight_decay_list:
    params = HParams(num_trees=300, max_depth=3, min_node_size=0, 
                     weight_decay=weight_decay, sub_samp=0.7, verbose=1, num_thresholds=20)
    params_list.append(params)
    
num_thresh_list = [10, 30, 50]
for num_thresh in num_thresh_list:
    params = HParams(num_trees=300, max_depth=3, min_node_size=0, 
                     weight_decay=0.6, sub_samp=0.7, verbose=1, num_thresholds=num_thresh)
    params_list.append(params)

max_depth_list = [1, 5, 7]
for max_depth in max_depth_list:
    params = HParams(num_trees=300, max_depth=max_depth, min_node_size=0, 
                     weight_decay=0.6, sub_samp=0.7, verbose=1, num_thresholds=20)
    params_list.append(params)


for params in params_list:
    print('Start {} {} {}'.format(params.max_depth, params.weight_decay, params.num_thresholds))

    _, model_log = gbrt(train_data=train_dataset.data, test_data=test_dataset.data, 
                   label_name=train_dataset.label_name, params=params)

    models_data[(depth, weight_decay, num_thresh)] = model_log

Start 3 0.6 10
Add tree number 1
Train mean loss is: 18.311351382073223
Test mean loss is: 17.171282341927597
Add tree number 2
Train mean loss is: 15.835926218882854
Test mean loss is: 14.905861743896436
Add tree number 3
Train mean loss is: 15.227401620666102
Test mean loss is: 14.444788554852737
Add tree number 4
Train mean loss is: 15.028155236157838
Test mean loss is: 14.285466304803784
Add tree number 5
Train mean loss is: 14.923569883740658
Test mean loss is: 14.210159060624514
Add tree number 6
Train mean loss is: 14.816602846503809
Test mean loss is: 14.132447816471794
Add tree number 7
Train mean loss is: 14.74000834190575
Test mean loss is: 14.122087220551563
Add tree number 8
Train mean loss is: 14.67290457468595
Test mean loss is: 14.121597220631301
Add tree number 9
Train mean loss is: 14.623013663232756
Test mean loss is: 14.076753625396405
Add tree number 10
Train mean loss is: 14.57011154320991
Test mean loss is: 14.056770090618375
Add tree number 11
Train mean loss is

Add tree number 87
Train mean loss is: 13.62102336240694
Test mean loss is: 13.938558494284718
Add tree number 88
Train mean loss is: 13.612790852645013
Test mean loss is: 13.938062749792694
Add tree number 89
Train mean loss is: 13.605249410085575
Test mean loss is: 13.935672065249149
Add tree number 90
Train mean loss is: 13.600777810560668
Test mean loss is: 13.93727867883587
Add tree number 91
Train mean loss is: 13.594099945542974
Test mean loss is: 13.939749592445882
Add tree number 92
Train mean loss is: 13.584687583723511
Test mean loss is: 13.937987046839405
Add tree number 93
Train mean loss is: 13.579869699253837
Test mean loss is: 13.937151536358682
Add tree number 94
Train mean loss is: 13.574406283086237
Test mean loss is: 13.938508582123651
Add tree number 95
Train mean loss is: 13.568467831594011
Test mean loss is: 13.956459367821937
Add tree number 96
Train mean loss is: 13.564807967706697
Test mean loss is: 13.964983620997428
Add tree number 97
Train mean loss is: 13.

Add tree number 172
Train mean loss is: 13.219899094488003
Test mean loss is: 14.019122117470069
Add tree number 173
Train mean loss is: 13.215628028657328
Test mean loss is: 14.018194284938538
Add tree number 174
Train mean loss is: 13.211941949068414
Test mean loss is: 14.016270715675828
Add tree number 175
Train mean loss is: 13.209145723705353
Test mean loss is: 14.024570619376826
Add tree number 176
Train mean loss is: 13.204506860807012
Test mean loss is: 14.022958584962605
Add tree number 177
Train mean loss is: 13.199852698303207
Test mean loss is: 14.023746365503396
Add tree number 178
Train mean loss is: 13.195204734990105
Test mean loss is: 14.025784156538345
Add tree number 179
Train mean loss is: 13.18935644242755
Test mean loss is: 14.014087983903305
Add tree number 180
Train mean loss is: 13.181779558428666
Test mean loss is: 14.013964628381611
Add tree number 181
Train mean loss is: 13.17920885776828
Test mean loss is: 14.014421159385154
Add tree number 182
Train mean l

Add tree number 257
Train mean loss is: 12.904022731409334
Test mean loss is: 14.091642651075729
Add tree number 258
Train mean loss is: 12.900066655363977
Test mean loss is: 14.092429314616522
Add tree number 259
Train mean loss is: 12.897233001208951
Test mean loss is: 14.095182808521432
Add tree number 260
Train mean loss is: 12.894773221632608
Test mean loss is: 14.094280782044077
Add tree number 261
Train mean loss is: 12.891762949862995
Test mean loss is: 14.08975180979072
Add tree number 262
Train mean loss is: 12.888975191350484
Test mean loss is: 14.087517592823072
Add tree number 263
Train mean loss is: 12.88676865034709
Test mean loss is: 14.086932387511872
Add tree number 264
Train mean loss is: 12.88391678354482
Test mean loss is: 14.087936809494966
Add tree number 265
Train mean loss is: 12.877745836031384
Test mean loss is: 14.09629198633844
Add tree number 266
Train mean loss is: 12.875088919249086
Test mean loss is: 14.095965966082957
Add tree number 267
Train mean los

Add tree number 43
Train mean loss is: 13.823305154632918
Test mean loss is: 13.861060170087034
Add tree number 44
Train mean loss is: 13.809143381752875
Test mean loss is: 13.875099792770364
Add tree number 45
Train mean loss is: 13.800149127393537
Test mean loss is: 13.879451972286539
Add tree number 46
Train mean loss is: 13.787011744964323
Test mean loss is: 13.897953805556662
Add tree number 47
Train mean loss is: 13.770939451529774
Test mean loss is: 13.88985884377317
Add tree number 48
Train mean loss is: 13.760789283091247
Test mean loss is: 13.89097460494898
Add tree number 49
Train mean loss is: 13.755787539291903
Test mean loss is: 13.884245046607587
Add tree number 50
Train mean loss is: 13.745661988424143
Test mean loss is: 13.892425784415272
Add tree number 51
Train mean loss is: 13.736698110013956
Test mean loss is: 13.888687383702464
Add tree number 52
Train mean loss is: 13.720204678115888
Test mean loss is: 13.89190229730896
Add tree number 53
Train mean loss is: 13.7

Add tree number 129
Train mean loss is: 13.155128106253647
Test mean loss is: 13.930408039323092
Add tree number 130
Train mean loss is: 13.148035458494075
Test mean loss is: 13.937260147392697
Add tree number 131
Train mean loss is: 13.143044126963986
Test mean loss is: 13.939921541016776
Add tree number 132
Train mean loss is: 13.138155216351194
Test mean loss is: 13.94022630650873
Add tree number 133
Train mean loss is: 13.134523327899519
Test mean loss is: 13.934943417869896
Add tree number 134
Train mean loss is: 13.13099047023953
Test mean loss is: 13.938093653905408
Add tree number 135
Train mean loss is: 13.125535661319054
Test mean loss is: 13.931718893616523
Add tree number 136
Train mean loss is: 13.11949396997046
Test mean loss is: 13.933216941979362
Add tree number 137
Train mean loss is: 13.114303085446213
Test mean loss is: 13.938839965932809
Add tree number 138
Train mean loss is: 13.11214348216107
Test mean loss is: 13.937044014432388
Add tree number 139
Train mean los

Add tree number 214
Train mean loss is: 12.771323529192838
Test mean loss is: 14.034359969032204
Add tree number 215
Train mean loss is: 12.76779287261882
Test mean loss is: 14.035680303845538
Add tree number 216
Train mean loss is: 12.764717893673062
Test mean loss is: 14.036461917823969
Add tree number 217
Train mean loss is: 12.760739149037924
Test mean loss is: 14.042636759775704
Add tree number 218
Train mean loss is: 12.758037933999983
Test mean loss is: 14.039856926205395
Add tree number 219
Train mean loss is: 12.75593737824303
Test mean loss is: 14.04539110099597
Add tree number 220
Train mean loss is: 12.751512930644502
Test mean loss is: 14.041544182961147
Add tree number 221
Train mean loss is: 12.748922808962615
Test mean loss is: 14.04392839829236
Add tree number 222
Train mean loss is: 12.744312333033326
Test mean loss is: 14.037692568474963
Add tree number 223
Train mean loss is: 12.743446297503422
Test mean loss is: 14.036846630149036
Add tree number 224
Train mean los

Add tree number 299
Train mean loss is: 12.48143561053437
Test mean loss is: 14.130799737497231
Add tree number 300
Train mean loss is: 12.477982548783894
Test mean loss is: 14.124907365830534
Start 3 0.6 50
Add tree number 1
Train mean loss is: 18.146589436436727
Test mean loss is: 17.046017532198412
Add tree number 2
Train mean loss is: 15.643547353577123
Test mean loss is: 14.74753095446224
Add tree number 3
Train mean loss is: 15.080993039402317
Test mean loss is: 14.322259574585312
Add tree number 4
Train mean loss is: 14.899598930632253
Test mean loss is: 14.182956792128689
Add tree number 5
Train mean loss is: 14.763590136535019
Test mean loss is: 14.093365825443847
Add tree number 6
Train mean loss is: 14.628001710857275
Test mean loss is: 14.006491735912917
Add tree number 7
Train mean loss is: 14.54842588173398
Test mean loss is: 13.950992737851744
Add tree number 8
Train mean loss is: 14.504394876923943
Test mean loss is: 13.927642542397162
Add tree number 9
Train mean loss 

Add tree number 85
Train mean loss is: 13.445622815981743
Test mean loss is: 13.912853941277284
Add tree number 86
Train mean loss is: 13.439471907787242
Test mean loss is: 13.914774628710283
Add tree number 87
Train mean loss is: 13.433526145991035
Test mean loss is: 13.910660846682543
Add tree number 88
Train mean loss is: 13.425581906860248
Test mean loss is: 13.908527126799012
Add tree number 89
Train mean loss is: 13.412074589234562
Test mean loss is: 13.911214352085143
Add tree number 90
Train mean loss is: 13.405884114561072
Test mean loss is: 13.920166249801596
Add tree number 91
Train mean loss is: 13.396127695999628
Test mean loss is: 13.929044848561064
Add tree number 92
Train mean loss is: 13.390953983446996
Test mean loss is: 13.922665097042227
Add tree number 93
Train mean loss is: 13.383298697360848
Test mean loss is: 13.927704478487762
Add tree number 94
Train mean loss is: 13.375819057417006
Test mean loss is: 13.912441340222045
Add tree number 95
Train mean loss is: 1

Add tree number 171
Train mean loss is: 12.945566136879002
Test mean loss is: 14.074369692612864
Add tree number 172
Train mean loss is: 12.940704285360965
Test mean loss is: 14.071074412208146
Add tree number 173
Train mean loss is: 12.937525470116558
Test mean loss is: 14.070389736532661
Add tree number 174
Train mean loss is: 12.93336345106652
Test mean loss is: 14.0691487821142
Add tree number 175
Train mean loss is: 12.931287228967959
Test mean loss is: 14.06211868103694
Add tree number 176
Train mean loss is: 12.928426564498183
Test mean loss is: 14.056805304791217
Add tree number 177
Train mean loss is: 12.924560654290252
Test mean loss is: 14.055070136044732
Add tree number 178
Train mean loss is: 12.920709579258073
Test mean loss is: 14.052984781879905
Add tree number 179
Train mean loss is: 12.915901269487021
Test mean loss is: 14.049120216031397
Add tree number 180
Train mean loss is: 12.91236872573922
Test mean loss is: 14.051971097703042
Add tree number 181
Train mean loss

# Deliverable 2 - Learning Curves with different parameter values

Hyperparams impact on error analysis. Here we plot train and test error as a function
of the number of trees in the ensemble. 

We compare plots for different values of Max Depth, Num Thresholds and Learning Rate.

Each time we vary one parameter, all other get the following values:
    <br>Learning Rate=0.6
    <br>Num Thresholds=20 
    <br>Min Node Size=0
    <br>Sub Sample=0.7 
    <br>Max Depth=3

All models were trained for 300 trees. 

In [ ]:
# Changing Max Depth
plt.figure(figsize=(10,10))
plt.suptitle('Training Plots, Changing Max Depth Values\n Learning Rate - 0.6, Num Thresholds - 20, Min Node Size - 0, Sub Sample - 0.7')
tups=[(1, 0.6, 20), (3, 0.6, 20), (5, 0.6, 20), (7, 0.6, 20)]
for ind, tup in enumerate(tups):
    plt.subplot(2, 2, ind+1)
    
    plt.title('max depth - {}'.format(tup[0]))

    tr = grid_search_data[tup]['train_loss']
    te = grid_search_data[tup]['test_loss']
    plt.plot(range(len(tr)), tr, c='red', label='Train')
    plt.plot(range(len(te)), te, c='blue', label='Test')

    plt.legend()

    if ind % 2 == 0:
        plt.ylabel('Mean Loss')
    if ind > 1:
        plt.xlabel('Number of Trees')    
plt.show()

# Changing Num Thresholds
plt.figure(figsize=(10,10))
plt.suptitle('Training Plots, Changing Num Thresholds Values\n Learning Rate - 0.6, Max Depth - 3, Min Node Size - 0, Sub Sample - 0.7')
tups=[(3, 0.6, 10), (3, 0.6, 20), (3, 0.6, 30), (3, 0.6, 50)]
for ind, tup in enumerate(tups):
    plt.subplot(2, 2, ind+1)
    
    plt.title('num thresholds - {}'.format(tup[2]))

    tr = grid_search_data[tup]['train_loss']
    te = grid_search_data[tup]['test_loss']
    plt.plot(range(len(tr)), tr, c='red', label='Train')
    plt.plot(range(len(te)), te, c='blue', label='Test')

    plt.legend()

    if ind % 2 == 0:
        plt.ylabel('Mean Loss')
    if ind > 1:
        plt.xlabel('Number of Trees')    
plt.show()


plt.figure(figsize=(10,10))
plt.suptitle('Training Plots, Changing Learning Rate Values\n Max Depth - 3, Num Thresholds - 20, Min Node Size - 0, Sub Sample - 0.7')
tups=[(3, 0.1, 20), (3, 0.3, 20), (3, 0.6, 20), (3, 0.9, 20)]
for ind, tup in enumerate(tups):
    plt.subplot(2, 2, ind+1)
    
    plt.title('Learning Rate - {}'.format(tup[1]))

    tr = grid_search_data[tup]['train_loss']
    te = grid_search_data[tup]['test_loss']
    plt.plot(range(len(tr)), tr, c='red', label='Train')
    plt.plot(range(len(te)), te, c='blue', label='Test')

    plt.legend()

    if ind % 2 == 0:
        plt.ylabel('Mean Loss')
    if ind > 1:
        plt.xlabel('Number of Trees')    
plt.show()

# Deliverable 3 - Training time plots

Hyperparams impact on runtime analysis. Here we Plot the time it takes to train a
boosted tree ensemble with 300 trees as function of hyperparameters. 
We chose to show plots varying the Max Depth and The Num Thresholds.


In [ ]:
tups=[(3, 0.6, 10), (3, 0.6, 20), (3, 0.6, 30), (3, 0.6, 50)]
